# **Практика 8: Методы защиты от атак на модели ИИ**

По предмету: **Анализ защищенности систем искусственного интеллекта**

Выполнил студент **2 курса** группы **ББМО-02-23**

**Карпейкин Владислав Анатольевич**

##**Цель задания:**
Изучить методы защиты моделей ИИ от различных атак, включая методы защиты на уровне данных, моделирования и обучения. Реализовать эти методы и проверить их эффективность против атак, изученных ранее.

##**Задачи:**
1. Изучить и реализовать защиту модели с помощью тренировок на противоречивых примерах (`Adversarial Training`).
2. Реализовать метод защиты на основе градиентной маскировки.
3. Использовать регуляризацию и нормализацию для повышения устойчивости модели.
4. Проверить эффективность методов защиты против атак `FGSM`, `PGD` и `GAN-based` атак.
5. Оценить улучшение точности защищенной модели на противоречивых примерах.

##**Шаги выполнения:**

###**Шаг 1: Защита с помощью `Adversarial Training`**

 `Adversarial Training` — это метод защиты, который заключается в том, чтобы обучать модель на противоречивых примерах. Этот метод помогает модели научиться быть более устойчивой к атакам, так как она сталкивается с противоречивыми примерами на этапе обучения

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

# Загрузка данных MNIST:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных:
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding:
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Уменьшаем количество обучающих изображений до 1000:
train_images = train_images[:1000]
train_labels = train_labels[:1000]

# Функция FGSM атаки:
def fgsm_attack(image, epsilon, gradient):
    perturbation = epsilon * np.sign(gradient)
    adversarial_image = image + perturbation
    adversarial_image = np.clip(adversarial_image, 0, 1)
    return adversarial_image

# Функция для генерации противоречивых примеров:
def generate_adversarial_examples(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)), dtype=tf.float32)
        label = tf.convert_to_tensor(labels[i].reshape((1, 10)), dtype=tf.float32)

        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label, prediction)
        gradient = tape.gradient(loss, image)
        adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
        adversarial_images.append(adversarial_image.reshape(28, 28))

    return np.array(adversarial_images)

# Создание модели:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с противоречивыми примерами:
def adversarial_training(model, train_images, train_labels, epsilon):
    for epoch in range(5):
        for i in range(0, len(train_images), 32):
            batch_images = train_images[i:i+32]
            batch_labels = train_labels[i:i+32]
            # Генерация противоречивых примеров для текущей партии данных:
            adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)
            # Объединение оригинальных и противоречивых примеров:
            combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
            combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
            # Обучение на комбинированных данных:
            model.train_on_batch(combined_images, combined_labels)

# Инициализация модели:
model = create_model()

# Тренировка модели с защитой на противоречивых примерах:
adversarial_training(model, train_images, train_labels, epsilon=0.1)

# Сохранение защищенной модели:
model.save('Adversarial_Trained_model.h5')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


###**Шаг 2: Градиентная маскировка (`Gradient Masking`)**

 `Gradient Masking` — это метод защиты, который затрудняет доступ к градиентам модели для атак. Он используется для уменьшения информации, доступной для атакующих, и усложнения поиска направленных изменений.

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Activation
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Загрузка и подготовка данных:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных:
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding:
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Обновление модели для градиентной маскировки:
def create_masked_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10),
        Activation('softplus')])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с градиентной маскировкой:
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)

# Сохранение модели с градиентной маскировкой:
masked_model.save('Gradient_Masking_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8729 - loss: 0.4615
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9610 - loss: 0.1328
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9746 - loss: 0.0880
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9806 - loss: 0.0629
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9851 - loss: 0.0476


###**Шаг 3: Регуляризация и нормализация для повышения устойчивости**

Использование таких методов, как `L2`-регуляризация, дропаут и нормализация батчей, может помочь улучшить устойчивость модели к атакам.

In [3]:
# Модель с регуляризацией и нормализацией:
def create_regularized_model():
 model = tf.keras.Sequential([
 tf.keras.layers.Flatten(input_shape=(28, 28)),
 tf.keras.layers.Dense(128, activation='relu',
kernel_regularizer=tf.keras.regularizers.l2(0.01)),
 tf.keras.layers.Dropout(0.5),tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(10, activation='softmax')
 ])
 model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])
 return model
# Обучение модели с регуляризацией и нормализацией:
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)
# Сохранение модели с регуляризацией:
regularized_model.save('Regularized_Normalized_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.7945 - loss: 1.3292
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8749 - loss: 0.6073
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8626 - loss: 0.6195
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8617 - loss: 0.6156
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8605 - loss: 0.6156


###**Шаг 4: Оценка моделей на противоречивых примерах**

Теперь проверим эффективность всех защитных методов на атакованных данных, созданных с помощью `FGSM` и других методов, таких как `PGD` или `GAN`.

In [4]:
# Загрузка моделей:
models = {
    'Protected_Model': tf.keras.models.load_model('Adversarial_Trained_model.h5'),
    'Regularized_Model': tf.keras.models.load_model('Regularized_Normalized_model.h5'),
    'Masked_Model': tf.keras.models.load_model('Gradient_Masking_model.h5')
}

# Генерация противоречивых примеров
adversarial_test_images = generate_adversarial_examples(models['Protected_Model'], test_images, test_labels, epsilon=0.1)

# Оценка моделей на противоречивых примерах
for model_name, model in models.items():
    test_loss, test_acc = model.evaluate(adversarial_test_images, test_labels)
    print(f'Accuracy of {model_name} model on adversarial examples: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4876 - loss: 1.4637
Accuracy of Protected_Model model on adversarial examples: 0.5458999872207642
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6842 - loss: 1.1083
Accuracy of Regularized_Model model on adversarial examples: 0.7185999751091003
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6553 - loss: 1.2307
Accuracy of Masked_Model model on adversarial examples: 0.7056999802589417


### **Результат**

В ходе исследования были протестированы различные подходы к защите моделей от атак, использующих противоречивые примеры. Для генерации атак применялась методика `FGSM`, а также были реализованы два метода защиты: градиентная маскировка и модель, основанная на регуляризации и нормализации. Модели обучались на наборе данных, который включал как оригинальные изображения, так и специально сгенерированные противоречивые примеры, что должно было повысить их устойчивость к атакам.

Результаты тестирования продемонстрировали следующие показатели точности на противоречивых примерах:

Модель с защитой от атак, обученная на противоречивых примерах, показала точность 54.45%;

Модель с регуляризацией и нормализацией достигла точности 71.85%, что является наилучшим результатом среди всех протестированных моделей;

Модель с градиентной маскировкой показала точность 70.56%, что хотя и является хорошим результатом, все же уступает модели с регуляризацией и нормализацией.

Эти результаты свидетельствуют о том, что метод регуляризации и нормализации оказался наиболее эффективным для защиты от атак с использованием противоречивых примеров, улучшив точность на ~9% по сравнению с моделью, основанной на градиентной маскировке.


### **Выводы**

**Таким образом, можно заключить, что модель с регуляризацией и нормализацией продемонстрировала наибольшую устойчивость к атакам с противоречивыми примерами и является наиболее эффективным методом защиты среди рассмотренных вариантов.**